In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
import os
import geopandas as gpd
from tobler.area_weighted import area_interpolate
from tobler.dasymetric import masked_area_interpolate
import math
import itertools
import optunity
import optunity.metrics
import sklearn.svm


from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata
import rasterio

In [2]:
path = os.getcwd()+ '\..\Dados vetoriais\Pontos amostrais_Piracicaba\Dados_227pts_original_06_03_2021.shp'
pontos_pira = gpd.read_file(path)

In [3]:
path2 = os.getcwd()+ '\..\Dados vetoriais\Piracicaba\Perimetro_Piracicaba.shp'
perimetro_pira = gpd.read_file(path2)


In [4]:
xizes = []
ypsolons = []

for geom in perimetro_pira.exterior:
    if(geom == None):
        continue
    xizes.append(geom.coords.xy[0])
    ypsolons.append(geom.coords.xy[1])

xizes = list(np.concatenate(xizes).flat)
ypsolons = list(np.concatenate(ypsolons).flat)

all_dots = [list(xy) for xy in zip(xizes, ypsolons)]

In [5]:
# split the data into featutes and target variable seperately
variaveis = pontos_pira.iloc[:, [1,2]].values # features set
argila = pontos_pira.iloc[:, 6].values # set of study variable

In [6]:
argila = argila.reshape(-1,1)

In [7]:
from sklearn.preprocessing import StandardScaler
StdS_X = StandardScaler()
StdS_y = StandardScaler()
variaveis = StdS_X.fit_transform(variaveis)
argila = StdS_y.fit_transform(argila)

In [8]:
# import the model
from sklearn.svm import SVR
# create the model object
model = SVR(kernel = 'rbf')
# fit the model on the data
model.fit(variaveis, argila.ravel())

SVR()

# Buscando o score
https://scikit-learn.org/stable/modules/model_evaluation.html

In [9]:
from sklearn.model_selection import cross_val_score
unused_scorings = ['neg_mean_squared_log_error','d2_tweedie_score','d2_absolute_error_score','d2_pinball_score','neg_mean_absolute_percentage_error','neg_mean_gamma_deviance','neg_mean_poisson_deviance']
scorings = ['explained_variance','max_error','neg_mean_absolute_error','neg_mean_squared_error','neg_root_mean_squared_error','neg_median_absolute_error','r2']
for score in scorings:
    print(score)
    print(cross_val_score(model, variaveis, argila.ravel(), cv=5, scoring=score))


explained_variance
[ 0.1034867   0.0868317   0.20283853 -0.44563926  0.23423109]
max_error
[-2.42540096 -3.34451686 -1.91205377 -3.00635168 -1.12948838]
neg_mean_absolute_error
[-0.75719364 -0.99767445 -0.82078122 -0.71642431 -0.44139174]
neg_mean_squared_error
[-0.879864   -1.72821432 -0.97090406 -0.86031131 -0.30395297]
neg_root_mean_squared_error
[-0.93801066 -1.31461565 -0.98534464 -0.92752968 -0.5513193 ]
neg_median_absolute_error
[-0.63653816 -0.70954236 -0.68517008 -0.51429859 -0.43805538]
r2
[ 0.09475738 -0.0691074   0.16341563 -0.45870487  0.22873687]


# R² Score

In [10]:
y_pred = model.predict(variaveis)
y_true = argila.ravel()

In [11]:
from sklearn.metrics import r2_score

transformed_predicted = StdS_y.inverse_transform(y_pred.reshape(-1,1))
transformed_actual = StdS_y.inverse_transform(y_true.reshape(-1,1))

r2_score(y_true, y_pred)

0.1999764321836187

# Mean Absolute Error

In [12]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_true, y_pred)

0.6533235590068922

# Mean Squared Error

In [13]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)

0.8000235678163813

# Median absolute error

In [14]:
from sklearn.metrics import median_absolute_error

median_absolute_error(y_true, y_pred)

0.4584071401094538

# Max Error

In [15]:
from sklearn.metrics import max_error

max_error(y_true, y_pred)

3.3089301611717126

# Explained Variance Score

In [16]:
from sklearn.metrics import explained_variance_score

explained_variance_score(y_true, y_pred)

0.2296977919858284